# 13.3 项目实施

## 13.3.1 获取所有Live

In [2]:
import requests

def scrapy(link):
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    r = requests.get(link, headers= headers)
    return (r.text)

link = "https://api.zhihu.com/lives/homefeed?includes=live"
html = scrapy(link)
print (html)

{"paging": {"is_end": false, "next": "https://api.zhihu.com/lives/homefeed?limit=10&offset=10", "previous": ""}, "data": [{"object_type": "live", "source_type": "member", "live": {"liked": false, "is_commercial": false, "video": {}, "seats": {"taken": 12964, "max": 500}, "id": "989811253094866944", "in_promotion": false, "fee": {"original_price": 999, "amount": 999, "unit": "RMB"}, "review": {"count": 1624, "status": "normal", "score": 4.78, "has_reviewed": false, "previous": {"status": "nonexistent", "score": 0}}, "cospeakers": [], "speaker": {"member": {"is_followed": false, "badge": [{"topics": [{"type": "topic", "id": 18012, "name": "医学"}], "type": "best_answerer", "description": "优秀回答者"}], "name": "马虫 医生", "url": "https://api.zhihu.com/people/7e8ca6a1f697eb8143a2107a4dd64ecf", "type": "people", "member_role": "svip", "user_type": "people", "headline": "敬业的外科医生 逗比的理想主义者", "avatar_url": "https://pic3.zhimg.com/v2-928bb40997855bf78aa795e2c725d25c_s.jpg", "is_following": false, "gende

In [3]:
import json
decodejson = json.loads(html)
next_page = decodejson['paging']['next']
is_end = decodejson['paging']['is_end']
print (next_page)
print (is_end)

https://api.zhihu.com/lives/homefeed?limit=10&offset=10
False


In [9]:
import requests
from pymongo import MongoClient
import json
import time
import random

#连接MongoDB
client = MongoClient('localhost',27017)
db = client.zhihu_database
collection = db.live

#定义爬虫函数
def scrapy(link):
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    r = requests.get(link, headers= headers)
    return (r.text)

link = "https://api.zhihu.com/lives/homefeed?includes=live"
is_end = False
#循环获取所有Live
while not is_end:
    html = scrapy(link)
    decodejson = json.loads(html)
    collection.insert_one(decodejson)
    
    link = decodejson['paging']['next']
    is_end = decodejson['paging']['is_end']
    print (link, is_end)
    time.sleep(random.randint(2,3) + random.random())

https://api.zhihu.com/lives/homefeed?limit=10&offset=10 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=20 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=30 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=40 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=50 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=60 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=70 False
https://api.zhihu.com/lives/homefeed?limit=10&offset=80 True


## 13.3.2 获取Live的听众

In [10]:
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.zhihu_database
collection = db.live

first_page = collection.find_one()
for each in first_page['data']:
    print (each['live']['id'])

989811253094866944
812015618365743104
870704471959822336
826057084528394240
860165089880330240
835121024906432512
897097999497437184
850711363683770368
1045284726512451584
927876522726027264


In [11]:
import requests
from pymongo import MongoClient
import json
import time
import random

client = MongoClient('localhost',27017)
db = client.zhihu_database
    
live_id = '989811253094866944'

def get_audience(live_id):
    headers = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.98 Safari/537.36'
    } 
    link = 'https://api.zhihu.com/lives/' + live_id + '/members?limit=10&offset=0'
    
    is_end = False
    while not is_end:
        r = requests.get(link, headers= headers)
        html = r.text
        decodejson = json.loads(html)
        decodejson['live_id'] = live_id
        db.live_audience.insert_one(decodejson)

        link = decodejson['paging']['next']
        is_end = decodejson['paging']['is_end']
        print(link, is_end)
        time.sleep(random.randint(2,3) + random.random())

get_audience(live_id)

https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=10 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=20 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=30 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=40 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=50 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=60 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=70 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=80 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=90 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=100 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=110 False
https://api.zhihu.com/lives/989811253094866944/members?limit=10&offset=120 False
https://api.zhihu.com/lives/989811253

KeyboardInterrupt: 

In [30]:
import requests
from pymongo import MongoClient
import json
import time
import random

client = MongoClient('localhost',27017)
db = client.zhihu_database

for each_page in db.live.find():
    for each in each_page['data']:
        live_id = each['live']['id']
        print (live_id)        
        get_audience(live_id)

840520148518592512
837669764146003968
837693085466107904
843124614065434624
842731562154987520
839869490635366400
839804209770692608
843110642910117888
840570499422650368
838157855663230976
840886183490912256
840879245080276992
837680104363679744
841344265740255232
842735083944308736
842735925179088896
839875793512456192
839859178712465408
835436065190993920
838163090607591424
841289257443274752
837679382431662080
838159504024682496
831457860721479680
840984548597714944
839864761947652096
833702327444725760
838388999964987392
835139866269474816
842740340824879104
819509716299436032
839869964864348160
837682828706746368
838405704879607808
827151989283635200
835125363007946752
836344346444967936
837373383242043392
835136598269112320
830380491126501376
835121024906432512
824618968198291456
831104976066609152


JSONDecodeError: Expecting value: line 1 column 1 (char 0)